# Bank Predictions

# Step 3: Develop

In [1]:
# Importing general packages
import pandas as pd
from imblearn.over_sampling import SMOTENC
import sys
sys.path.insert(0, './scripts/')

from scripts.helpers import load_data
from scripts import Preprocessing

In [ ]:
# Importing all sci-kit learn packages
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
bank_data_full = load_data('./raw_data/', 'bank-full.csv', sep=';')

In [29]:
cat_cols = [x for x in bank_data_full.columns if bank_data_full[x].dtype == object]
num_cols = [x for x in bank_data_full.columns if bank_data_full[x].dtype == float]
unsure_cols = [x for x in bank_data_full.columns if bank_data_full[x].dtype == int]
unsure_cols

['age', 'duration', 'campaign', 'pdays', 'previous']

In [34]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import PolynomialFeatures

le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)
lb = LabelBinarizer()

y_enc = le.fit_transform(bank_data_full['y'])
# data = ohe.fit_transform(bank_data_full[cat_cols], )
# lb.fit_transform(bank_data_full['y'])
# labels = ohe.get_feature_names()
# pd.DataFrame(data, columns=labels)

pf = PolynomialFeatures()
pf.fit_transform(bank_data_full[num_cols])
pd.Index(pf.get_feature_names(bank_data_full[num_cols].columns))

Index(['1', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'emp.var.rate^2', 'emp.var.rate cons.price.idx',
       'emp.var.rate cons.conf.idx', 'emp.var.rate euribor3m',
       'emp.var.rate nr.employed', 'cons.price.idx^2',
       'cons.price.idx cons.conf.idx', 'cons.price.idx euribor3m',
       'cons.price.idx nr.employed', 'cons.conf.idx^2',
       'cons.conf.idx euribor3m', 'cons.conf.idx nr.employed', 'euribor3m^2',
       'euribor3m nr.employed', 'nr.employed^2'],
      dtype='object')

## Engineering Features
* Ensure data is ready for modeling
* Create any new features to enhance the model

## Creating Models
* Creating and tuning models

## Testing Models
* Doing a 5-fold cross validation on models

## Selecting the Best Model
* Selecting the model with the lowest error as the "production" model